In [18]:
import numpy as np
from numba import float64,int64,njit, prange

from vect_rcantile import RE, R2D, CE, EPSILON

In [2]:
@njit('float64[:](float64, float64)', parallel=True, fastmath=True)
def truncate_lnglat(lng, lat):
    result = np.zeros((2), dtype=float64)
    if lng > 180.0:
        result[0] = 180.0
    elif lng < -180.0:
        result[0] = -180.0
    else:
        result[0] = lng

    # Truncate latitudes: similarly, directly modifying the output array
    if lat > 90.0:
        result[1] = 90.0
    elif lat < -90.0:
        result[1] = -90.0
    else:
        result[1] = lat
    return result

In [3]:
@njit('float64[:](float64, float64, boolean)', parallel=True, fastmath=True)
def xy(lng, lat, truncate=False):
    result = np.zeros((2), dtype=float64)
    if truncate:
        lng, lat = truncate_lnglat(lng, lat)

    result[0] = RE * np.radians(lng)

    if lat <= -90:
        result[1] = -np.inf
    elif lat >= 90:
        result[1] = np.inf
    else:
        result[1] = RE * np.log(np.tan((np.pi * 0.25) + (0.5 * np.radians(lat))))

    return result

In [4]:
@njit('float64[:](float64, float64, boolean)', parallel=True, fastmath=True)
def lnglat(x, y, truncate=False):
    result = np.zeros((2), dtype=float64)
    result[0], result[1] = (
        x * R2D / RE,
        ((np.pi * 0.5) - 2.0 * np.arctan(np.exp(-y / RE))) * R2D,
    )
    if truncate:
        result[0], result[1] = truncate_lnglat(result[0], result[1])
    return result

In [5]:
@njit('float64[:](int64, int64, int64)', parallel=True, fastmath=True)
def ul(x, y, zoom):
    result = np.zeros((2), dtype=float64)
    Z2 = 2 ** zoom
    lon_deg = x / Z2 * 360.0 - 180.0
    lat_rad = np.arctan(np.sinh(np.pi * (1 - 2 * y / Z2)))
    lat_deg = np.degrees(lat_rad)
    result[0] = lon_deg
    result[1] = lat_deg
    return result

In [6]:
@njit('float64[:](int64,int64,int64)', parallel=True, fastmath=True)
def bounds(x, y, zoom):
    result = np.zeros((4), dtype=float64)
    Z2 = 2 ** zoom
    result[0] = x / Z2 * 360.0 - 180.0
    result[1] = np.degrees(np.arctan(np.sinh(np.pi * (1 - 2 * (y + 1) / Z2))))
    result[2] = (x + 1) / Z2 * 360.0 - 180.0
    result[3] = np.degrees(np.arctan(np.sinh(np.pi * (1 - 2 * y / Z2))))
    return result

In [9]:
@njit('float64[:](int64,int64,int64)', parallel=True, fastmath=True)
def xy_bounds(x, y, zoom):
    result = np.zeros((4), dtype=float64)
    tile_size = CE / 2 ** zoom
    result[0] = x * tile_size - CE / 2
    result[1] = result[0] + tile_size
    result[2] = CE / 2 - y * tile_size
    result[3] = result[2] - tile_size
    return result

In [13]:
@njit('float64[:](float64,float64)', parallel=True, fastmath=True)
def _xy(lng, lat):
    result = np.zeros((2), dtype=float64)
    result[0] = lng / 360.0 + 0.5
    result[1] = 0.5 - 0.25 * np.log((1.0 + np.sin(np.radians(lat))) / (1.0 - np.sin(np.radians(lat)))) / np.pi
    return result

In [20]:
@njit('int64[:](float64,float64,int64)', parallel=True, fastmath=True)
def tile(lng, lat, zoom):
    result = np.zeros((3), dtype=int64)
    x = lng / 360.0 + 0.5
    y = 0.5 - 0.25 * np.log((1.0 + np.sin(np.radians(lat))) / (1.0 - np.sin(np.radians(lat)))) / np.pi
    Z2 = 2 ** zoom
    result[2] = zoom
    if x <= 0:
        result[0] = 0
    elif x >= 1:
        result[0] = np.int64(Z2 - 1)
    else:
        result[0] = np.int64(np.floor((x + EPSILON) * Z2))

    if y <= 0:
        result[1] = 0
    elif y >= 1:
        result[1] = np.int64(Z2 - 1)
    else:
        result[1] = np.int64(np.floor((y + EPSILON) * Z2))
    return result

In [33]:
print(tile(55.679134, 37.263656,10))

[670 397  10]


In [48]:
@njit('int64[:](int64,int64,int64)', parallel=True, fastmath=True)
def quadkey(xtile,ytile,zoom):
    """Get the quadkey of a tile

    Parameters
    ----------
    tile : Tile or sequence of int
        May be be either an instance of Tile or 3 ints, X, Y, Z.

    Returns
    -------
    str

    """
    result = np.zeros((zoom), dtype=int64)
    for i in range(zoom):
        digit = 0
        bit = zoom - i - 1
        if (xtile >> bit) & 1:
            digit += 1
        if (ytile >> bit) & 1:
            digit += 2
        result[i] = digit
    return result